In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import ResNet50
from keras_tuner import RandomSearch

last_path = [[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[0,1,1],[1,0,1],[1,1,1]]
prefix = [f"{i:03d}" for i in range(505, 513)]
prefix_to_label = dict(zip(prefix, last_path))

def process_dataset(root_folder):
    image_paths = []
    label_data = []

    for roots, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.jpg'):
                # 파일 이름 분석을 위해 숫자만 추출
                prefix = file[0:3]
                
                # 접두사에 따른 레이블 할당
                label = prefix_to_label.get(prefix)
                
                # 유효한 레이블이 있는 경우에만 리스트에 추가
                if label is not None:
                    image_paths.append(os.path.join(roots, file))
                    label_data.append(label)
    
    return image_paths, label_data

# 각각의 데이터셋에 대해 함수를 호출
train_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\training'
valid_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\validation'
test_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\test'

train_image_paths, train_label_data = process_dataset(train_folder)
valid_image_paths, valid_label_data = process_dataset(valid_folder)
test_image_paths, test_label_data = process_dataset(test_folder)

# 필요에 따라 결과를 확인하거나 다른 처리를 수행
print(len(train_image_paths), len(train_label_data))
print(len(valid_image_paths), len(valid_label_data))
print(len(test_image_paths), len(test_label_data))

['E:\\AI\\dataset_skeleton_sep\\face\\BicycleCrunch\\training\\BicycleCrunch_505\\505-Z106_C-0000001.jpg', 'E:\\AI\\dataset_skeleton_sep\\face\\BicycleCrunch\\training\\BicycleCrunch_505\\505-Z106_C-0000004.jpg', 'E:\\AI\\dataset_skeleton_sep\\face\\BicycleCrunch\\training\\BicycleCrunch_505\\505-Z106_C-0000005.jpg', 'E:\\AI\\dataset_skeleton_sep\\face\\BicycleCrunch\\training\\BicycleCrunch_505\\505-Z106_C-0000006.jpg', 'E:\\AI\\dataset_skeleton_sep\\face\\BicycleCrunch\\training\\BicycleCrunch_505\\505-Z106_C-0000008.jpg', 'E:\\AI\\dataset_skeleton_sep\\face\\BicycleCrunch\\training\\BicycleCrunch_505\\505-Z106_C-0000010.jpg', 'E:\\AI\\dataset_skeleton_sep\\face\\BicycleCrunch\\training\\BicycleCrunch_505\\505-Z106_C-0000011.jpg', 'E:\\AI\\dataset_skeleton_sep\\face\\BicycleCrunch\\training\\BicycleCrunch_505\\505-Z106_C-0000013.jpg', 'E:\\AI\\dataset_skeleton_sep\\face\\BicycleCrunch\\training\\BicycleCrunch_505\\505-Z106_C-0000014.jpg', 'E:\\AI\\dataset_skeleton_sep\\face\\BicycleC

In [2]:
def resize_img(image_paths):
    images_resized = []  # 리사이즈된 이미지를 저장할 리스트
    for image_path in image_paths:
        image = cv2.imread(image_path)  # 각 이미지 경로로부터 이미지를 읽음
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 색상 변환
        image_resized = cv2.resize(image, (128, 128))  # 이미지 리사이즈
        images_resized.append(image_resized)  # 결과 리스트에 추가
    images_resized = np.array(images_resized) / 255.0  # numpy 배열로 변환 및 정규화
    return images_resized

train_image_resized = resize_img(train_image_paths)
valid_image_resized = resize_img(valid_image_paths)
test_image_resized = resize_img(test_image_paths)

train_label_data = np.array(train_label_data)
valid_label_data = np.array(valid_label_data)
test_label_data = np.array(test_label_data)

print('done')

done


In [3]:
# 모델 구성    
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    
for layer in base_model.layers:
    base_model.trainable = False
for layer in base_model.layers[-9:]:
    base_model.trainable = True
    
# 모델 구성    
model = models.Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(3, activation='sigmoid'))


In [4]:
# def build_model(hp):
#     base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    
#     for layer in base_model.layers:
#         base_model.trainable = False
#     for layer in base_model.layers[-9:]:
#         base_model.trainable = True
        
#     # 모델 구성    
#     model = models.Sequential()
#     model.add(base_model)
#     model.add(layers.Flatten())
#     model.add(layers.Dense(units=hp.Int('units', min_value=128, max_value=512, step=128), activation = 'relu'))
#     model.add(layers.Dense(3, activation='sigmoid')) # 멀티라벨 분류를 위한 sigmoid 활성화 함수 사용
        
#     model.compile(optimizer=optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-3, sampling='LOG')),
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])

#     return model


In [5]:
# tuner = RandomSearch(
#     build_model,
#     objective='val_accuracy',
#     max_trials=5,  # 시도할 하이퍼파라미터 조합의 최대 개수
#     executions_per_trial=1,  # 각 하이퍼파라미터 설정을 평가하기 위해 모델을 훈련시킬 횟수
#     directory=r'E:\model\ResNet_RandomSearch',  # 튜닝 세션의 결과를 저장할 디렉토리 이름
#     project_name='BicycleCrunch_multilabel_01', # 프로젝트 이름    
# )
# tuner.search_space_summary()

In [6]:
earlystopping = EarlyStopping(monitor='val_loss', patience=7, mode='min', verbose=1)

# tuner.search(train_image_resized, 
#              train_label_data, 
#              validation_data=(valid_image_resized, valid_label_data), 
#              epochs=25, 
#              batch_size=32,
#              callbacks=[earlystopping])


# # 최적의 하이퍼파라미터 가져오기
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# # 최적의 하이퍼파라미터로 모델 빌드
# model = tuner.hypermodel.build(best_hps)

In [7]:
# 모든 최적 하이퍼파라미터 출력
# for hp in best_hps.values:
#     print(f"The optimal value for {hp} is {best_hps.get(hp)}")

model.compile(optimizer=optimizers.Adam(learning_rate=0.0002),
              loss=['binary_crossentropy'],
              metrics=['accuracy'])
    
# 모델 훈련
model.fit(train_image_resized, 
          train_label_data, 
          validation_data=(valid_image_resized, valid_label_data), 
          epochs=25, 
          batch_size=32,
          callbacks=[earlystopping])

# 모델 평가 (테스트 데이터셋)
test_loss, test_acc = model.evaluate(test_image_resized, test_label_data)
print('\n테스트 정확도:', test_acc)


Epoch 1/25
247/247 [==============================] - 29s 95ms/step - loss: 0.6406 - accuracy: 0.3330 - val_loss: 0.6994 - val_accuracy: 0.1248
Epoch 2/25
247/247 [==============================] - 23s 92ms/step - loss: 0.5747 - accuracy: 0.3388 - val_loss: 0.7025 - val_accuracy: 0.1248
Epoch 3/25
247/247 [==============================] - 23s 91ms/step - loss: 0.4985 - accuracy: 0.4004 - val_loss: 0.7589 - val_accuracy: 0.1248
Epoch 4/25
247/247 [==============================] - 24s 96ms/step - loss: 0.4420 - accuracy: 0.4495 - val_loss: 0.6145 - val_accuracy: 0.2815
Epoch 5/25
247/247 [==============================] - 22s 89ms/step - loss: 0.3707 - accuracy: 0.4727 - val_loss: 0.7641 - val_accuracy: 0.3016
Epoch 6/25
247/247 [==============================] - 22s 87ms/step - loss: 0.3031 - accuracy: 0.5084 - val_loss: 0.7183 - val_accuracy: 0.5559
Epoch 7/25
247/247 [==============================] - 22s 87ms/step - loss: 0.2410 - accuracy: 0.5465 - val_loss: 0.5395 - val_accuracy:

In [8]:
# 모델 예측
predictions = model.predict(test_image_resized)

# 임계값 설정 (예: 0.5)
threshold = 0.5
predictions_binary = (predictions > threshold).astype(int)

# 각 레이블에 대한 정확도 계산
accuracy_per_label = np.mean(predictions_binary == test_label_data, axis=0)

# 각 레이블별 정확도 출력
for i, accuracy in enumerate(accuracy_per_label):
    print(f"레이블 {i}의 정확도: {accuracy}")

# 전체 정확도도 여전히 중요할 수 있으므로, 이를 계산합니다.
overall_accuracy = np.mean(predictions_binary == test_label_data)
print(f"전체 정확도: {overall_accuracy}")

53/53 [==============================] - 2s 19ms/step
레이블 0의 정확도: 0.8227622999407231
레이블 1의 정확도: 0.8257261410788381
레이블 2의 정확도: 0.8672199170124482
전체 정확도: 0.8385694526773365
